# Python AST（抽象構文木）入門ノートブック

このノートブックは **Pythonの抽象構文木（AST）** を、手を動かして学べるように作られています。

内容:
- `ast.parse` の基本
- ノードの走査（`ast.walk`, `NodeVisitor`）
- 変換（`NodeTransformer`）でコードを書き換える
- `ast.unparse` でソースへ戻す（Python 3.9+）
- ASTで作る簡易「安全評価器」
- 応用: 自動ログ挿入 / 演算子回数の可視化

すべてのセルにコメントを入れてあります。Google Colab / VS Code / Kaggle などクラウド環境で実行できます。

## 0. 事前準備
- このノートブックは標準ライブラリのみで動きます（`ast`）。
- Pythonバージョン: 3.9+ 推奨（`ast.unparse`を使うため）。3.8以下では `astor` を使う方法もあります。

In [ ]:
import sys, ast, textwrap, math, operator
print(sys.version)
# 出力されたバージョンを確認してください。3.9 以上だと ast.unparse が利用できます。

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


## 1. `ast.parse` の基本
`ast.parse(source, mode)` は、Pythonコード文字列を「構文木」に変換します。

In [ ]:
code = """
x = 10
y = x * 2 + 5
def add(a, b):
    return a + b
print(add(y, 3))
"""

tree = ast.parse(code, mode="exec")
print(type(tree), tree)  # <class '_ast.Module'> など

# ast.dump で構造を文字列化（indent=4 で見やすく）
print(ast.dump(tree, indent=4))

<class 'ast.Module'> <ast.Module object at 0x7e57084b9f50>
Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store())],
            value=Constant(value=10)),
        Assign(
            targets=[
                Name(id='y', ctx=Store())],
            value=BinOp(
                left=BinOp(
                    left=Name(id='x', ctx=Load()),
                    op=Mult(),
                    right=Constant(value=2)),
                op=Add(),
                right=Constant(value=5))),
        FunctionDef(
            name='add',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a'),
                    arg(arg='b')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Return(
                    value=BinOp(
                        left=Name(id='a', ctx=Load()),
                        op=Add(),
              

## 2. ノードを歩く: `ast.walk` と `NodeVisitor`
`ast.walk` は全ノードを網羅的にたどれます。`NodeVisitor` はクラスベースで、特定ノードタイプに対して処理を分けられます。

In [ ]:
# walk で全てのノードタイプを数える例
from collections import Counter
counts = Counter(type(n).__name__ for n in ast.walk(tree))
print(counts)

# NodeVisitor で関数定義名を収集
class FuncCollector(ast.NodeVisitor):
    def __init__(self):
        self.func_names = []
    def visit_FunctionDef(self, node):
        self.func_names.append(node.name)
        self.generic_visit(node)  # 子ノードも探索

collector = FuncCollector()
collector.visit(tree)
collector.func_names

Counter({'Name': 8, 'Load': 6, 'Constant': 4, 'BinOp': 3, 'Assign': 2, 'Call': 2, 'Store': 2, 'Add': 2, 'arg': 2, 'Module': 1, 'FunctionDef': 1, 'Expr': 1, 'arguments': 1, 'Return': 1, 'Mult': 1})


['add']

## 3. ASTでコードを書き換える: `NodeTransformer`
ここでは、`x * 2` の形を `x << 1` （左シフト）に置き換えるお遊び変換を示します（意味は同等ではない場合がありますが、パターン変換の例として）。

In [ ]:
class TimesTwoToShift(ast.NodeTransformer):
    def visit_BinOp(self, node):
        # まず子を変換
        self.generic_visit(node)
        # 形: <something> * 2
        if isinstance(node.op, ast.Mult) and isinstance(node.right, ast.Constant) and node.right.value == 2:
            # x * 2 -> x << 1 に変換
            return ast.BinOp(left=node.left, op=ast.LShift(), right=ast.Constant(1))
        return node

tree2 = ast.parse(code)
new_tree = TimesTwoToShift().visit(tree2)
ast.fix_missing_locations(new_tree)  # 位置情報を補完（unparseに必要）

print(ast.unparse(new_tree))  # 3.9+ でソースコードへ逆変換

x = 10
y = (x << 1) + 5

def add(a, b):
    return a + b
print(add(y, 3))


## 4. 関数呼び出しへ自動ログ挿入
すべての関数定義に `print(f"<func> called")` を先頭に差し込みます。AST変換はこうした**横断的関心事（ロギング、検証、トレース）**に便利です。

In [ ]:
class InjectLogger(ast.NodeTransformer):
    def visit_FunctionDef(self, node):
        self.generic_visit(node)
        log_stmt = ast.parse(f'print("[LOG] {node.name} called")').body[0]
        node.body.insert(0, log_stmt)
        return node

tree3 = ast.parse(code)
logged = InjectLogger().visit(tree3)
ast.fix_missing_locations(logged)
print(ast.unparse(logged))

# 実行して確認
compiled = compile(logged, filename="<ast>", mode="exec")
exec_globals = {}
exec(compiled, exec_globals)  # 実行時に [LOG] add called が出ればOK

x = 10
y = x * 2 + 5

def add(a, b):
    print('[LOG] add called')
    return a + b
print(add(y, 3))
[LOG] add called
28


## 5. 安全な数式評価（ミニ版）
外部入力を `eval` せず、**許可演算子のみ**を評価する安全評価器の最小例です。

In [ ]:
BIN_OPS = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow,
}
UNARY_OPS = {ast.UAdd: operator.pos, ast.USub: operator.neg}
ALLOWED = {"pi": math.pi, "e": math.e, "sqrt": math.sqrt}

def safe_eval(expr: str):
    node = ast.parse(expr, mode="eval").body
    def _ev(n):
        if isinstance(n, ast.Constant) and isinstance(n.value, (int, float)):
            return n.value
        if isinstance(n, ast.BinOp) and type(n.op) in BIN_OPS:
            return BIN_OPS[type(n.op)](_ev(n.left), _ev(n.right))
        if isinstance(n, ast.UnaryOp) and type(n.op) in UNARY_OPS:
            return UNARY_OPS[type(n.op)](_ev(n.operand))
        if isinstance(n, ast.Name) and n.id in ALLOWED and not callable(ALLOWED[n.id]):
            return ALLOWED[n.id]
        if isinstance(n, ast.Call) and isinstance(n.func, ast.Name) and n.func.id in ALLOWED:
            args = [_ev(a) for a in n.args]
            return ALLOWED[n.func.id](*args)
        raise ValueError("未許可の構文/演算です")
    return _ev(node)

print(safe_eval("1 + 2 * 3"))
print(safe_eval("sqrt(2) * pi"))
try:
    print(safe_eval("__import__('os').system('echo NG')"))
except Exception as e:
    print("安全に拒否されました:", e)


7
4.442882938158366
安全に拒否されました: 未許可の構文/演算です


## 6. 演算子の回数を集計して可視化（テキスト）
ソースコード内で使われている演算子（`+`, `-`, `*`, `**`など）の登場回数を数えて、改善の観点（複雑度のヒント）として使えます。

In [ ]:
sample = """
def calc(a, b):
    x = a + b
    y = x * 2 - a ** 2
    return (y / 3) + (a - b)
"""
op_names = {ast.Add: '+', ast.Sub: '-', ast.Mult: '*', ast.Div: '/', ast.Pow: '**'}

counts = Counter()
for n in ast.walk(ast.parse(sample)):
    if isinstance(n, ast.BinOp) and type(n.op) in op_names:
        counts[op_names[type(n.op)]] += 1

print("演算子の出現回数:")
for k, v in counts.items():
    print(f"  {k}: {v}")

演算子の出現回数:
  +: 2
  -: 2
  *: 1
  **: 1
  /: 1


## 7. 既存コードへ型ヒントを自動付与（簡易）
引数や戻り値が数値と仮定して、`-> float` を関数定義へ付与する例です（実務では型推論が必要になるので、これはあくまで雰囲気）。

In [ ]:
class AddReturnFloatHint(ast.NodeTransformer):
    def visit_FunctionDef(self, node):
        self.generic_visit(node)
        # 既に戻り値注釈がある場合は触らない
        if node.returns is None:
            node.returns = ast.Name(id="float")
        # 引数にも float を雑に付与（デモ目的）
        for a in node.args.args:
            if a.annotation is None:
                a.annotation = ast.Name(id="float")
        return node

typed = AddReturnFloatHint().visit(ast.parse(sample))
ast.fix_missing_locations(typed)
print(ast.unparse(typed))

## 8. まとめ
- ASTを使うと **解析（Visitor）**と**変換（Transformer）** を分離して安全にコード操作ができます。
- `ast.unparse`（3.9+）でラウンドトリップ（AST→ソース）も簡単。
- ちょっとした静的解析や自動変換、教育用ツールの試作に最適です。